In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, Tensor
import torch.optim as optim

from src.utils import load_data

### Load data

In [2]:
X, y, k = load_data()
print('X-shape: {}, y-shape: {}'.format(X.shape, y.shape)) 
X.head()

X-shape: (2095, 35), y-shape: (2095,)


,youth_pop_5to18,commute_priv_vehicle,med_hhincome,avg_ann_daily_traffic,fragment_index,TotalPop,Men,Women,Hispanic,White,...,Transit,Walk,OtherTransp,WorkAtHome,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,894,0.634634,74837,8682.0,-3.061385,5403,2659,2744,75.8,2.3,...,38.6,2.9,0.0,0.0,2308,80.8,16.2,2.9,0.0,7.7
1,1158,0.494977,77991,16917.0,-2.915361,5915,2896,3019,62.7,3.6,...,44.6,1.4,0.5,2.1,2675,71.7,25.3,2.5,0.6,9.5
2,1120,0.422405,32354,22712.0,-0.227456,5879,2558,3321,65.1,1.6,...,45.5,8.6,1.6,1.7,2120,75.0,21.3,3.8,0.0,8.7
3,281,0.199795,34635,124767.0,3.029461,2591,1206,1385,55.4,9.0,...,63.9,3.0,2.4,6.2,1083,76.8,15.5,7.7,0.0,19.2
4,1998,0.286795,23423,10219.0,0.165237,8516,3301,5215,61.1,1.6,...,68.2,4.3,1.0,0.0,2508,71.0,21.3,7.7,0.0,17.2


In [3]:
# standardize input features.
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

In [4]:
# Prepare data for model.
X = torch.tensor(np.array(X)).float()
y = torch.tensor(y).float()

### Define autoencoder

In [5]:
class Autoencoder(nn.Module):
    
    """An autoencoder"""
    
    
    def __init__(self, input_shape, latent_features):
        super(Autoencoder, self).__init__()
        
        self.input_shape = input_shape
        self.latent_features = latent_features
        self.observation_features = np.prod(input_shape)

        # Inference Network
        # Encode the observation `x` into the parameters of the posterior distribution
        # `q_\phi(z|x) = N(z | \mu(x), \sigma(x)), \mu(x),\log\sigma(x) = h_\phi(x)`
        self.encoder = nn.Sequential(
            nn.Linear(in_features=self.observation_features, out_features=32),
            nn.LeakyReLU(),
            nn.Linear(in_features=32, out_features=24),
            nn.Tanh(),
            nn.Linear(in_features=24, out_features=latent_features)
        )
        
        # Generative Model
        # Decode the latent sample `z` into the parameters of the observation model
        # `p_\theta(x | z) = \prod_i B(x_i | g_\theta(x))`
        self.decoder = nn.Sequential(
            nn.Linear(in_features=latent_features, out_features=24),
            nn.Tanh(),
            nn.Linear(in_features=24, out_features=32),
            nn.LeakyReLU(),
            nn.Linear(in_features=32, out_features=self.observation_features)
        )        

    def forward(self, x):
        """Run encoder and decoder, and output latent space together with reconstruction."""
        
        # Latent space
        h = self.encoder(x)

        y = self.decoder(h)
        
        return h, y
    
    def encode(self, x):
        """Run encoder and decoder, and output latent space together with reconstruction."""
        
        # Latent space
        h = self.encoder(x)
        
        return h


In [6]:
latent_features = 18
ae = Autoencoder(X.shape[1], latent_features)
print(ae)

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=35, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=24, bias=True)
    (3): Tanh()
    (4): Linear(in_features=24, out_features=18, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=18, out_features=24, bias=True)
    (1): Tanh()
    (2): Linear(in_features=24, out_features=32, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=32, out_features=35, bias=True)
  )
)


In [7]:
# Load model.
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')
ae.to(device)
print("Running on device", device)

Running on device cuda


Create data loader for batch handling

In [8]:
from torch.utils.data import DataLoader, TensorDataset

X_train = X[:1800]
X_test = X[1800:]

# Output currently not used.
y_train = y[:1800]
y_test = y[1800:]

data_train = TensorDataset(X_train, y_train)

batch_size = 64
train_loader = torch.utils.data.DataLoader(
    data_train, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
)

### Train network

In [ ]:
from tqdm import tqdm

# Train model.
epochs = 200
eval_every = 1

criterion = nn.MSELoss()
optimizer = optim.Adam(ae.parameters(), lr=0.0001)
train_losses = []
test_losses = []

t = tqdm(range(epochs))
for epoch in t:

    # Training.
    ae.train()    

    for x, _ in train_loader:

        x = x.to(device)
        
        h, x_recon = ae(x)
        
        optimizer.zero_grad()

        # compute training reconstruction loss
        loss = criterion(x_recon, x)
        
        # compute accumulated gradients
        loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
    ae.eval()
    if epoch % eval_every == 0:
        # Train evaluation.
        x = X_train.to(device)
        h, x_recon = ae(x)
        # compute training reconstruction loss
        train_loss = criterion(x_recon, x).item()
        
        x = X_test.to(device)
        h, x_recon = ae(x)
        # compute test reconstruction loss
        test_loss = criterion(x_recon, x).item()
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)
    
    # display the epoch training loss
    t.set_description("epoch : {}/{}, train_loss = {:.4f}, test_loss = {:.4f}".format(epoch + 1, epochs, train_loss, test_loss))
    t.refresh() # to show immediately the update
    

epoch : 169/200, train_loss = 0.3145, test_loss = 0.4239:  84%|██████████████████▌   | 169/200 [05:24<01:03,  2.03s/it]

In [ ]:
plt.figure()
plt.plot(train_losses, label='Train loss')
plt.plot(test_losses, label='Test loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.show()

Plot example of reconstruction of the 35 features

In [ ]:
plt.show()
plt.plot(X[0], label='Orig')
plt.plot(ae(X[0].to(device))[1].detach().cpu().numpy(), label='Recon')
plt.legend()
plt.show()